# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [58]:
import pandas as pd
import numpy as np
from scipy.linalg import solve_discrete_are


In [59]:
# YOUR CHANGES HERE

A = pd.read_csv('model-A.tsv', sep='\t', index_col=0).values
B = pd.read_csv('model-B.tsv', sep='\t', index_col=0).values
Q = pd.read_csv('cost-Q.tsv', sep='\t', index_col=0).values
R = pd.read_csv('cost-R.tsv', sep='\t', index_col=0).values

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"Q shape: {Q.shape}")
print(f"R shape: {R.shape}")


P = solve_discrete_are(A, B, Q, R)

# Compute the control matrix K
K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

print(f"\nControl matrix K:\n{K}")
print(f"K shape: {K.shape}")

A shape: (3, 3)
B shape: (3, 1)
Q shape: (3, 3)
R shape: (1, 1)

Control matrix K:
[[0.33445985 1.30445413 1.85813088]]
K shape: (1, 3)


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [60]:
# YOUR CHANGES HERE

if K.ndim == 1:
    K = K.reshape(1, -1)
K_df = pd.DataFrame(K, columns=['x', 'v', 'a'], index=['u'])
K_df.to_csv('control-K-intended.tsv', sep='\t', float_format='%.6f')
print("\nSaved control-K-intended.tsv")



Saved control-K-intended.tsv


Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [69]:
# YOUR CHANGES HERE


log_data = pd.read_csv('qc-log.tsv', sep='\t')
states = log_data[['x', 'v', 'a']].values.astype(np.float64)
K = K.astype(np.float64)
u_check = -(K @ states.T).reshape(-1)
u_check = np.around(u_check, decimals=6)


Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [79]:
# YOUR CHANGES HERE
check_df = pd.DataFrame({
    'u_check': u_check
}, index=range(len(u_check)))

check_df.index.name = 'index'
check_df.to_csv('qc-check.tsv', sep='\t', float_format='%.6f')
print("\nSaved qc-check.tsv")


Saved qc-check.tsv


Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [63]:
# YOUR CHANGES HERE

from sklearn.linear_model import LinearRegression

X = states  
y = -logged_actions  

reg = LinearRegression(fit_intercept=False)
reg.fit(X, y)

K_actual = reg.coef_ 
K_actual = K_actual.reshape(1, -1)

print(f"\nIntended K:\n{K}")
print(f"\nActual K (reverse engineered):\n{K_actual}")
print(f"Difference:\n{K - K_actual}")



Intended K:
[[0.33445985 1.30445413 1.85813088]]

Actual K (reverse engineered):
[[0.34043755 1.30012023 1.95011696]]
Difference:
[[-0.0059777   0.0043339  -0.09198608]]


Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [64]:
# YOUR CHANGES HERE

if K_actual.ndim == 1:
    K_actual = K_actual.reshape(1, -1)
K_actual_df = pd.DataFrame(K_actual, columns=['x', 'v', 'a'], index=['u'])
K_actual_df.to_csv('control-K-actual.tsv', sep='\t', float_format='%.6f')


Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [65]:
# YOUR CHANGES HERE
train_data = pd.read_csv('qc-train.tsv', sep='\t')
print(f"Training data shape: {train_data.shape}")
print(f"Columns: {train_data.columns.tolist()}")

state_t = train_data[['x', 'v', 'a']].values[:-1]
actions = train_data['u'].values[:-1].reshape(-1, 1)
state_next = train_data[['x', 'v', 'a']].values[1:]

print(f"\nstate_t shape: {state_t.shape}")
print(f"actions shape: {actions.shape}")
print(f"state_next shape: {state_next.shape}")

X_dyn = np.hstack([state_t, actions])

A_new = np.zeros((3, 3))
B_new = np.zeros((3, 1))

for i in range(3):
    y_dyn = state_next[:, i]
    reg_dyn = LinearRegression(fit_intercept=False)
    reg_dyn.fit(X_dyn, y_dyn)
    coeffs = reg_dyn.coef_
    A_new[i, :] = coeffs[:3]
    B_new[i, 0] = coeffs[3]

print(f"\nEstimated A matrix:\n{A_new}")
print(f"\nEstimated B matrix:\n{B_new}")

Training data shape: (100, 5)
Columns: ['index', 'x', 'v', 'a', 'u']

state_t shape: (99, 3)
actions shape: (99, 1)
state_next shape: (99, 3)

Estimated A matrix:
[[ 1.00000000e+00  1.10000000e+00  9.99200722e-16]
 [ 5.82012590e-17  9.00000000e-01  9.50000000e-01]
 [-9.31378031e-17  5.55111512e-16  1.10000000e+00]]

Estimated B matrix:
[[-5.55111512e-17]
 [-1.00000000e-02]
 [ 9.00000000e-01]]


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [66]:
# YOUR CHANGES HERE
A_new_df = pd.DataFrame(A_new, columns=['x', 'v', 'a'], index=['x', 'v', 'a'])
B_new_df = pd.DataFrame(B_new, columns=['u'], index=['x', 'v', 'a'])
A_new_df.to_csv('model-A-new.tsv', sep='\t', float_format='%.6f')
B_new_df.to_csv('model-B-new.tsv', sep='\t', float_format='%.6f')



Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.